# Earthquake seismology with DAS

#### Todo List:
1. Reference earthquakes by Evid rather than dataframe index
2. Make a map of all of the detected events
3. Try some xcorr

In [1]:
import h5py
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import numpy as np
import datetime
import pandas as pd
from dasquakes import sintela_to_datetime, open_sintela_file, local_earthquake_quicklook, data_wrangler, haversine
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from libcomcat.search import search
from libcomcat.dataframes import get_summary_data_frame

ERROR 1: PROJ: proj_create_from_database: Open of /home/velgueta/.conda/envs/python-obspy/share/proj failed


### Get a list of events from USGS

In [2]:
events = search(starttime = datetime.datetime(2022, 3, 17, 0, 0), 
                endtime   = datetime.datetime.now(),
                minmagnitude=3.0,
                maxradiuskm=600, 
                latitude=47.60, 
                longitude=-122.33)

print("Returned %s events" % len(events))
 

Returned 31 events


### Choose an event to analyze

In [3]:
event_df = get_summary_data_frame(events)
event_df = event_df.sort_values(by=['time'],ascending=False)
event_df


#event_df.iloc[0::]

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance
30,us6000jli2,2023-02-05 04:39:10.395,"41 km NW of Kimberley, Canada",49.928200,-116.394700,10.000,3.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,268
29,us6000jkrr,2023-02-01 14:11:22.720,"158 km WNW of Port Orford, Oregon",43.069600,-126.391600,10.000,3.80,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,222
28,us6000jk08,2023-01-28 16:10:51.411,None,48.999000,-128.559200,10.000,4.00,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,247
27,us7000j3g5,2022-12-28 08:07:48.265,"266 km SW of Tofino, Canada",47.840200,-128.923300,12.086,3.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,234
26,us7000j3f8,2022-12-28 08:06:54.980,"267 km SW of Tofino, Canada",47.804700,-128.908400,9.158,3.80,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,222
25,us7000ishe,2022-11-26 03:50:17.526,"18 km NW of Tofino, Canada",49.271400,-126.092300,33.380,4.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,527
24,us7000iqsk,2022-11-18 15:42:45.032,off the coast of Oregon,43.374100,-126.635600,10.000,4.40,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,298
23,us7000imha,2022-11-04 14:16:58.043,Off the coast of Oregon,43.401300,-127.099500,10.000,3.40,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,178
22,us7000im59,2022-11-03 09:57:33.841,Off the coast of Oregon,43.327300,-126.840700,10.000,3.50,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,189
21,us7000ikhh,2022-10-26 15:18:19.752,"169 km W of Langlois, Oregon",43.173500,-126.500500,10.000,3.20,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,158


In [4]:
for index,this_event in event_df.iterrows():
    
    this_id = this_event['id']
    
    this_event_new = event_df[event_df.id==this_id]
    
    latitude_loc = this_event_new['latitude']
    longitude_loc = this_event_new['longitude']
    latitude_ref = 47.60
    longitude_ref = -122.33
    
    #distance = haversine(longitude_loc,latitude_loc,longitude_ref,latitude_ref)
    
    #if distance < 500:
    #    continue
    
    print(this_event['id'])
    
    stitle=f"M {this_event_new['magnitude'].to_numpy()[0]}, "\
           f"{this_event_new['location'].to_numpy()[0]}, "\
           f"{str(this_event_new['time'].to_numpy()[0])[0:-10]} UTC"
      

        
    cable = 'whidbey'
    record_length = 7 #minutes
    t0 = this_event_new['time'].iloc[0]
    
    prefix, network_name, datastore = data_wrangler(cable,record_length,t0)
        
    try:
        data,dates,attrs = open_sintela_file(prefix,
                                             t0,
                                             datastore,
                                             number_of_files=record_length,
                                             verbose=True)
        x_max=data.shape[1] * attrs['SpatialSamplingInterval']
        
        low_cut = 2
        hi_cut = 10

        b,a = butter(2,(low_cut,hi_cut),'bp',fs=attrs['MaximumFrequency']*2)
        data_filt = filtfilt(b,a,data,axis=0)
        try:
            client = Client("IRIS")

            t1 = UTCDateTime( t0.strftime("%Y-%m-%dT%H:%M:%S") )
            dt = datetime.timedelta(minutes=record_length)
            t2 = UTCDateTime( (t0+dt).strftime("%Y-%m-%dT%H:%M:%S") )

            st = client.get_waveforms("UW", "NOWS", "--", "HNN", t1, t2, attach_response=True)
            # st.decimate(5, strict_length=False, no_filter=False) #everything should be at 40Hz now
            st2 = client.get_waveforms("IU", "COR", "00", "HH1", t1, t2, attach_response=True)

            pre_filt = (low_cut/2, low_cut, hi_cut, hi_cut*2)
            st.remove_response(output='VEL', pre_filt=pre_filt)
            st2.remove_response(output='VEL', pre_filt=pre_filt)
        
            filename = f"earthquakes_distances600km/"+this_id
            local_earthquake_quicklook(dates,data_filt,st,st2,x_max,
                           stitle,filename=filename,das_vmax=0.1,
                           network_name=network_name,
                           skip_seismograms=True)
        except Exception as e:
             print(str(e))
        continue
    except ValueError:
        continue
    #except :
       # continue
       # timestamps = []
        #for d in dates:
         #   if type(d) == datetime.datetime:
         #       timestamps.append(int((d.timestamp()-25200)*10**6))
          #  else:
           #     timestamps.append(d)
        #g=h5py.File(filename+'.h5','w')
        #g.create_dataset('datachannel',data=data_filt)
        #g.create_dataset('time',data=np.array(timestamps))
        #g.attrs.update(attrs)
       
        #g.close()
        
    
        # x_max=data.shape[1] * attrs['SpatialSamplingInterval']
        
   # low_cut = 2
    #hi_cut = 10

    #b,a = butter(2,(low_cut,hi_cut),'bp',fs=attrs['MaximumFrequency']*2)
    #data_filt = filtfilt(b,a,data,axis=0)
        
    #filename = f"earthquakes_figures"
    #local_earthquake_quicklook(dates,data_filt,x_max,
                      #     stitle,filename=filename,das_vmax=0.02,
                       #    network_name=network_name,
                        #   skip_seismograms=True)


us6000jli2
Searching for files matching: /data/data6/whidbey/whidbey_2023-02-05_04-39*h5
us6000jkrr
Searching for files matching: /data/data6/whidbey/whidbey_2023-02-01_14-11*h5
us6000jk08
Searching for files matching: /data/data6/whidbey/whidbey_2023-01-28_16-10*h5
us7000j3g5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-07*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-08*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-09*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-10*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-11*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-12*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-13*h5
us7000j3f8
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-06*h5
Searching for files matching: /data/data6/whidbey/whidbey_2022-12-28_08-07*h5
Searching

KeyboardInterrupt: 

# Data frame for earthquakes with WhibdeyCable

In [ ]:
tf=['Nocablerunning','False','False','False','False','Nostationstocompare','True','Nostationstocompare','False','False','True/False','True','False','True/False','False','False','True','False','False','False','False','False','True','Nostationstocompare','Nostationstocompare','False','True','Nostationstocompare','Nostationstocompare','False','Nostationstocompare']
tfn=np.array(tf).T
dataf = np.array([event_df.id, event_df.location, event_df.magnitude,tf]).T

df3 = pd.DataFrame(dataf,columns=['Id', 'Distance', 'Magnitude','Detected by Das'])
df3

